<a href="https://colab.research.google.com/github/rachanabramhane/assignment/blob/main/covid_scan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
from google.colab import drive

In [29]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
covid_path = ("/content/drive/MyDrive/COVID")
noncovid_path = ("/content/drive/MyDrive/non-COVID")

In [16]:
import os
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [39]:
import nibabel as nib

from scipy import ndimage


def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume


In [34]:
covid_path = [
    os.path.join(os.getcwd(), "/content/drive/MyDrive/COVID", x)
    for x in os.listdir("/content/drive/MyDrive/COVID")
]

noncovid_path = [
    os.path.join(os.getcwd(), "/content/drive/MyDrive/non-COVID", x)
    for x in os.listdir("/content/drive/MyDrive/non-COVID")
]

print("CT scans with normal lung tissue: " + str(len(noncovid_path)))
print("CT scans with abnormal lung tissue: " + str(len(covid_path)))

CT scans with normal lung tissue: 1229
CT scans with abnormal lung tissue: 1273


In [42]:
# Read and process the scans.
# Each scan is resized across height, width, and depth and rescaled.
covid_scans = np.array([process_scan(path) for path in covid_path])
noncovid_scans = np.array([process_scan(path) for path in noncovid_path])

# For the CT scans having presence of covid
# assign 1, for the noncovid ones assign 0.
covid_labels = np.array([1 for _ in range(len(covid_scans))])
noncovid_labels = np.array([0 for _ in range(len(noncovid_scans))])

# Split data in the ratio 70-30 for training and validation.
x_train = np.concatenate((covid_scans[:70], noncovid_scans[:70]), axis=0)
y_train = np.concatenate((covid_labels[:70], noncovid_labels[:70]), axis=0)
x_val = np.concatenate((covid_scans[70:], noncovid_scans[70:]), axis=0)
y_val = np.concatenate((covid_labels[70:], noncovid_labels[70:]), axis=0)
print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)

ImageFileError: ignored